In [8]:
from pyspark.sql import SparkSession
from pyspark import SparkConf,  SparkContext

In [9]:
import numpy as np, os, shutil, json, time
import pickle as pkl
from datetime import datetime
from nlp import preprocessAndGetTokens
# from paperAbstracts import processPaperAbstract

In [10]:
from fileUtils import load, pickleLoader, dump, saveByPartition

In [11]:
driver_memory = '2g'
executor_memory = '2g'
scheduler_mode = "FAIR"
files_maxPatitionsBytes = '500Mo'

pyspark_submit_args = ' --driver-memory {0} --executor-memory {1} pyspark-shell'\
                                .format(driver_memory, executor_memory)
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args
pyspark_submit_args

' --driver-memory 2g --executor-memory 2g pyspark-shell'

In [12]:
# conf = SparkConf().setAll([
# #     ('spark.executor.memory', '2g'),
#      ('spark.app.name', 'pLDA'), 
#      ('spark.executor.cores', '4'), 
#      ('spark.cores.max', '4'), 
#      #('spark.driver.memory','3g'),
# #     ('spark.scheduler.mode', 'FAIR'),
#     ('spark.files.maxPartitionBytes', '500Mo')])

In [13]:
conf = SparkConf().setAll([
     ('spark.app.name', 'pLDA'), 
     ('spark.executor.cores', '4'), 
     ('spark.cores.max', '4'), 
    ('spark.scheduler.mode', 'FAIR'),
    ('spark.files.maxPartitionBytes', '500Mo')])

In [14]:
spark = SparkContext(conf = conf)

In [15]:
# spark = SparkSession(context).sparkContext

In [16]:
spark._conf.getAll()

[('spark.files.maxPartitionBytes', '500Mo'),
 ('spark.scheduler.mode', 'FAIR'),
 ('spark.driver.host', '192.168.0.41'),
 ('spark.executor.id', 'driver'),
 ('spark.executor.cores', '4'),
 ('spark.app.id', 'local-1549466503181'),
 ('spark.cores.max', '4'),
 ('spark.driver.port', '42223'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.name', 'pLDA'),
 ('spark.driver.memory', '2g'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

In [17]:
# spark = SparkSession.builder\
#     .master("local[*]") \
#     .appName("lda") \
#     .getOrCreate().sparkContext

In [18]:
nbPartitions = 20

In [19]:
def processPaperAbstract(abstract):
    return np.array(preprocessAndGetTokens(abstract))

In [20]:
def LoadPaperAbstract(docstr, trunc = 1500):
    doc = json.loads(docstr)
    return (doc["id"], doc["paperAbstract"][:trunc])

In [21]:
# data = spark.textFile("sample-S2-records").map(LoadPaperAbstract)
data = spark.textFile('/home/nerk/Downloads/s2-corpus-46.gz')\
        .repartition(nbPartitions)\
                .map(LoadPaperAbstract)

In [22]:
data.getNumPartitions()

20

In [24]:
%%time

data.map(lambda x : len(x[1])).take(10)

[488, 0, 0, 0, 971, 0, 1500, 1132, 0, 0]

In [26]:
# %%time
# data.take(2) 

In [27]:
%%time

corpus = data.mapValues(processPaperAbstract)\
                    .filter(lambda x : len(x[1]) > 0)\
                    .saveAsPickleFile("corpus/corpus-46", 5000)
#corpus.take(1)

CPU times: user 50.6 ms, sys: 7.79 ms, total: 58.4 ms
Wall time: 9min 51s


In [28]:
# corpus.map(lambda x: len(x[1])).distinct().collect()

# Patitionning

In [29]:
# def randomPartitionner(x, nbPartitions):
#     return (np.random.choice(nbPartitions),x)

In [30]:
corpus = spark.pickleFile("corpus/corpus-46/part-0000*" )
corpus.getNumPartitions()

40

In [31]:
corpus = corpus.repartition(nbPartitions)
corpus.getNumPartitions()

20

In [32]:
# corpus2 = corpus.map(lambda x  : randomPartitionner(x, nbPartitions))\
#             .partitionBy(nbPartitions)

In [ ]:
%%time
corpus.take(1)

# Make the vocabularies

In [ ]:
import importlib, builder
importlib.reload(builder)

In [ ]:
from builder  import makeVocabularies, makeVocabulariesFolder, getUniqueWords

In [ ]:
makeVocabulariesFolder()

In [ ]:
%%time
uniqueWordsByPartition = corpus.mapPartitionsWithIndex(getUniqueWords).collect()

print( "Number of unique words by partition : %s"%str([len(u) for u in uniqueWordsByPartition]))

In [ ]:
%%time

makeVocabularies([ w[1] forw in  uniqueWordsByPartition]) # Build and save the vocabularies

# Make docMaps

In [ ]:
# import builder, importlib
# importlib.relaod(builder)

In [ ]:
from builder import makeDocsMaps, makeDocsMapsFolder

In [31]:
makeDocsMapsFolder()

In [32]:
%%time

corpus.mapPartitionsWithIndex(makeDocsMaps).collect()

CPU times: user 8.3 ms, sys: 0 ns, total: 8.3 ms
Wall time: 658 ms


['docMap 0 successfully built',
 'docMap 1 successfully built',
 'docMap 2 successfully built',
 'docMap 3 successfully built',
 'docMap 4 successfully built',
 'docMap 5 successfully built',
 'docMap 6 successfully built',
 'docMap 7 successfully built',
 'docMap 8 successfully built',
 'docMap 9 successfully built',
 'docMap 10 successfully built',
 'docMap 11 successfully built',
 'docMap 12 successfully built',
 'docMap 13 successfully built',
 'docMap 14 successfully built',
 'docMap 15 successfully built',
 'docMap 16 successfully built',
 'docMap 17 successfully built',
 'docMap 18 successfully built',
 'docMap 19 successfully built']

# Test if vocabularies and docMaps are correctly built

In [33]:
# vocabSize = len(load("matrix/vocabulary/vocabAll"))
# print("vocabSize : %d "%vocabSize)

In [34]:
# ndocs00 = len(load("matrix/docsMap/docs__0000__"))
# print("Number of docs in docs00 : %d"%ndocs00)

# As voacabularies & docMaps was well built, let's load them

In [40]:
%%time
vocabAll = load("matrix/vocabulary/vocabAll")

vocabs = [load("matrix/vocabulary/vocab__%04d__"%ind) for ind in range(nbPartitions)] 

CPU times: user 1.3 s, sys: 47 ms, total: 1.35 s
Wall time: 1.35 s


In [41]:
%%time
from builder import loadDocsAll
docsAll = loadDocsAll(nbPartitions)

docs = [load("matrix/docsMap/docs__%04d__"%ind) for ind in range(nbPartitions)] 

CPU times: user 20.7 ms, sys: 4.09 ms, total: 24.8 ms
Wall time: 23.9 ms


In [42]:
%%time
nbDocs = list(map(len, docs))
nbVocabs = list(map(len, vocabs))

CPU times: user 16 µs, sys: 2 µs, total: 18 µs
Wall time: 22.6 µs


# Encode corpus : using ids instead of doc full text

In [71]:
import builder, importlib
importlib.reload(builder)
from builder  import encodeAddTopics

In [72]:
%%time 
corpus2.take(1)

CPU times: user 8.07 ms, sys: 0 ns, total: 8.07 ms
Wall time: 75.5 ms


[('66d6270fad8c662721b87b91c8f23148184eb710',
  array(['result', 'lie', 'level', 'suffici', 'greater', 'mabp', 'washout',
         'control', 'seizur', 'case', 'enhanc', 'critic', 'fail', 'associ',
         'must', 'effect', 'within', 'valu', 'cerebr', 'restor', 'rise',
         'provid', 'requir', 'rang', 'cbf', 'progress', 'whether', 'attenu',
         'elev', 'deliveri', 'mark', 'later', 'oxidationreduct', 'maintain',
         'metabol', 'confirm', 'seizureassoci', 'increas', 'oxygen',
         'measur', 'determin', 'decreas', 'also', 'cortic', 'adequ',
         'defin', 'aa', 'meet', 'focal', 'rose', 'cytochrom', 'conclud',
         'exceed', 'relat', 'accompani', 'inspir', 'earli', 'chang', 'rat',
         'repetit', 'ventil'], dtype='<U15'))]

In [58]:
# Notice that all the words have been encoded into symbolic ids
corpus2 = corpus2.mapPartitionsWithIndex(lambda ind, part : encode(ind, part, docs, vocabs))

# End data preparation by adding random topics and saving 

In [59]:
nbTopics = 10

In [60]:
corpTop = corpus2.map(lambda x : (x[0], x[1], np.random.choice(nbTopics, len(x[1]) )))

In [61]:
# %%time

# corpTop.mapPartitionsWithIndex(saveByPartition).collect() # Save bacthes by partition

In [62]:
%%time

corpTop.take(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 15.0 failed 1 times, most recent failure: Lost task 0.0 in stage 15.0 (TID 107, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/nerk/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in main
    process()
  File "/home/nerk/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 367, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/nerk/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 390, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/nerk/anaconda3/lib/python3.7/site-packages/pyspark/rdd.py", line 1354, in takeUpToNumLeft
    yi

# Here the ML part

# Define some parameters

In [ ]:
nbVocabAll = len(vocabAll)
alpha = 10
beta = 10

In [ ]:
from builder  import init

In [ ]:
# from builder import makeConfig, updateConfig, get_now
# makeConfig(id = "all", countWordsUpdated = {str(ind):False for ind in range(nbPartitions)}, time = get_now())

# Training

In [ ]:
from fileUtils import saveAsPickleFile
from model import pldaMap0
from builder import updateCountWordsAll, init

In [ ]:
# pldaMap(0, 1, alpha, beta, len(vocabAll), nbTopics)

In [ ]:
%%time

t0 = time.time()
rdd = corpTop
init(corpTop, vocabs, nbDocs, nbVocabs, len(vocabAll), nbTopics)
for i in range(100):
    rdd = rdd.mapPartitionsWithIndex(lambda ind, part : pldaMap0(ind, part, alpha, beta, nbVocabAll, nbTopics),
                       preservesPartitioning= True )
    saveAsPickleFile(rdd)
    rdd = spark.pickleFile("pickle/")
    updateCountWordsAll()
print("Time : {}".format(time.time() - t0))

# Post-training analysis

In [ ]:
%%time
cl = 0
subdoc = 2
countDocs = load("matrix/countDocs/docs__%04d__"%subdoc)
countDocs[:10]

In [ ]:
topics = countDocs.argmax(1)
v = np.where(topics == cl)
countDocs[v]

In [ ]:
dks = np.array( list(docs[subdoc].items()))
cluster = dks[v]
cluster

In [88]:
corpus.filter(lambda x : np.isin(x[0], cluster[:, 0])).collect()

[('676415ecd7062992875cdca0e7c015eb0790885f',
  array(['result', 'mixtur', 'ultraviolet', 'lean', 'vari', 'direct',
         'apertur', 'relationship', 'sensit', 'contain', 'tissu',
         'fragment', 'diamet', 'type', 'photomultipli', 'onto', 'cm',
         'fluoresc', 'fiber', 'content', 'particl', 'degre', 'reticular',
         'angl', 'curvilinear', 'emiss', 'muscl', 'circular', 'comminut',
         'greatest', 'sampl', 'ratio', 'fewest', 'weak', 'ligament',
         'adipos', 'nm', 'matrix', 'tendon', 'low', 'measur', 'surfac',
         'emitt', 'intact', 'correl', 'strong', 'meatgristl', 'iii', 'peak',
         'light', 'less', 'collagen', 'slight', 'gristl',
         'longissimusshank', 'elast', 'two', 'dri', 'monochrom', 'mix',
         'tube', 'around', 'area', 'chang', 'meat', 'high', 'trial'],
        dtype='<U16')),
 ('a8673869f70b6a41be1022fc951bdbb0e57eade6',
  array(['result', 'clear', 'human', 'older', 'greater', 'aerial',
         'control', 'coastal', 'season', 'vee